In [ ]:
# !pip3 install botometer
import tqdm
import re
import pandas as pd
import random
import pprint
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import botometer
import numpy
import json
import seaborn as sns
import pickle as pkl

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
RegB= loadPickle('Regular_Before_py37.pkl')
RegA= loadPickle('Regular_After_py37.pkl')
regularAfter= list(set(list(RegA[RegA['user.id_str'].isin(RegB['user.id_str'])].copy()['user.id_str'])))
RegB, RegA= None, None

ResB= loadPickle('Restored_Before_py37.pkl')
ResA= loadPickle('Restored_After_py37.pkl')
restoredAfter= list(set(list(ResA[ResA['user.id_str'].isin(ResB['user.id_str'])==True].copy()['user.id_str'])))
ResB, ResA= None, None

In [ ]:
resultRegular, resultRestored= {}, {}

In [ ]:
remaining_users_regular= [int(i) for i in list(set(regularAfter).difference(set([str(j) for j in list(resultRegular.keys())])))]

In [ ]:
len(remaining_users_regular)

In [ ]:
rapidapi_key= ""
twitter_app_auth= {}
bom= botometer.Botometer(wait_on_ratelimit= True, rapidapi_key= rapidapi_key, **twitter_app_auth)

In [ ]:
for screen_name, result in tqdm.tqdm(bom.check_accounts_in(remaining_users_regular)):
    resultRegular[screen_name]= result

In [ ]:
with open("bot_regular_after.json", 'w') as fout:
    json_dumps_str= json.dumps(resultRegular, indent= 2)
    print(json_dumps_str, file= fout)

In [ ]:
remaining_users_restored= [int(i) for i in list(set(restoredAfter).difference(set([str(j) for j in list(resultRestored.keys())])))]

In [ ]:
len(remaining_users_restored)

In [ ]:
for screen_name, result in tqdm.tqdm(bom.check_accounts_in(remaining_users_restored)):
    resultRestored[screen_name]= result

In [ ]:
len(resultRegular), len(resultRestored)

In [ ]:
with open("bot_restored_after.json", 'w') as fout:
    json_dumps_str= json.dumps(resultRestored, indent= 2)
    print(json_dumps_str, file= fout)

In [ ]:
with open("bot_regular_after.json", 'r') as fout:
    regularAfterBot= json.load(fout)

In [ ]:
with open("bot_restored_after.json", 'r') as fout:
    restoredAfterBot= json.load(fout)

In [ ]:
df= {'id_str': [], 'astroturf_universal': [], 'overall_universal': [], 'Users': []}

In [ ]:
for i in restoredAfterBot:
    temp= restoredAfterBot[i]
    if temp.get('error') is None:
        df['id_str'].append(temp['user']['user_data']['id_str'])
        df['overall_universal'].append(temp['cap']['universal'])
        df['astroturf_universal'].append(temp['raw_scores']['universal']['astroturf'])
        df['Users'].append('Regular (Timeline)')

In [ ]:
for i in regularAfterBot:
    temp= regularAfterBot[i]
    if temp.get('error') is None:
        df['id_str'].append(temp['user']['user_data']['id_str'])
        df['overall_universal'].append(temp['cap']['universal'])
        df['astroturf_universal'].append(temp['raw_scores']['universal']['astroturf'])
        df['Users'].append('Restored (Timeline)')

In [ ]:
languages_df= pd.DataFrame.from_dict(df)

In [ ]:
attr= 'overall_universal'
temp_cdf= sns.ecdfplot(
    data= languages_df.sort_values(by= attr),
    x= attr,
    hue= 'Users',
    stat= 'proportion',
    palette= [
        "#6B7280",
        "#374151"
    ]
)

In [ ]:
attr= 'astroturf_universal'
temp_cdf= sns.ecdfplot(
    data= languages_df.sort_values(by= attr),
    x= attr,
    hue= 'Users',
    stat= 'proportion',
    palette= [
        "#374151",
        "#6B7280"
    ]
)